In [ ]:
class logit_probit:
    import scipy as sp
    import pandas as pd
    import numpy as np
    import warnings
    import statsmodels as sm
    warnings.filterwarnings("ignore")
    # By this command, all annoying warning signs would be ignored.
    
    def __init__(self,x,y,distr,Add_intercept):  
        if Add_intercept=='True':
            self.x=sm.add_constant(x)
        elif Add_intercept=='False':
            self.x=x
        # The Matrix of X, one can choose whether to add a constant term.
        self.y=y 
        # Vector of y.
        if distr == 'probit':
            self.distr = sp.stats.norm
        elif distr == 'logit':
            self.distr = sp.stats.logistic # Here, one can choose which model (logit or probit) to use.
        
    def loglikelihood(self,parm):
        mu_vec=np.array((-np.inf,0,np.inf)) 
        # Since logit and probit are the special cases of ordered logit and probit, I use the similar method by setting the threhold (only zero here) to estimate them.
        inner=self.distr.cdf(mu_vec[[self.y.astype('int')+1]]-np.dot(self.x,parm),0,1)-self.distr.cdf(mu_vec[[self.y.astype('int')]]-np.dot(self.x,parm),0,1)
        return -np.sum(np.log(inner))
    
    def params(self):
        def loglikelihood(parm):
            mu_vec=np.array((-np.inf,0,np.inf)) 
            inner=self.distr.cdf(mu_vec[[self.y.astype('int')+1]]-np.dot(self.x,parm),0,1)-self.distr.cdf(mu_vec[[self.y.astype('int')]]-np.dot(self.x,parm),0,1)
            return -np.sum(np.log(inner))
             
        return sp.optimize.fmin_bfgs(loglikelihood,np.array([0,0,0]))
    
    def std(self):
        def loglikelihood(parm):
            mu_vec=np.array((-np.inf,0,np.inf)) 
            inner=self.distr.cdf(mu_vec[[self.y.astype('int')+1]]-np.dot(self.x,parm),0,1)-self.distr.cdf(mu_vec[[self.y.astype('int')]]-np.dot(self.x,parm),0,1)
            return -np.sum(np.log(inner))
        self.res=sp.optimize.minimize(loglikelihood,np.array([0,0,0]), method='bfgs')
        # Since I need to use the Hessian matrix to get the information matrix, I switch to the scipy.minimize moduel, which can easily get it.
        self.std=np.sqrt(np.diagonal(self.res.hess_inv))
        
        return np.sqrt(np.diagonal(self.res.hess_inv))
    
    def summary(self):
        def loglikelihood(parm):
            mu_vec=np.array((-np.inf,0,np.inf)) 
            inner=self.distr.cdf(mu_vec[[self.y.astype('int')+1]]-np.dot(self.x,parm),0,1)-self.distr.cdf(mu_vec[[self.y.astype('int')]]-np.dot(self.x,parm),0,1)
            return -np.sum(np.log(inner))
        self.res=sp.optimize.minimize(loglikelihood,np.array([0,0,0]), method='bfgs')
        self.std=np.sqrt(np.diagonal(self.res.hess_inv))
        self.z=self.res.x/self.std # Calculate the Z-value.
        self.ci_up=1.96*self.std+self.res.x # Upper bound of the 95% CI.
        self.ci_low=-1.96*self.std+self.res.x # Lower bound of the 95% CI.
        self.data = {'coef.':self.res.x,'std.errors':self.std,'z-value':self.z,'[0.025':self.ci_low,'0.975]':self.ci_up} 
        row=['_cons']
        for i in range(self.x.shape[1]-1):
            row.append('x'+str(i+1))
        # By this for loop, the row indices will be x1,x2,...,xN, where N is the number of explanatory variables.
        return pd.DataFrame(self.data, index=row) 
    
    def bootstrap(self,niter, method):
        
        def logl(parm,x,y): # Here, I rewrite the log-likelihood function by adding x & y as the parameters in this function because in the below part I need to use this function in bootstrap estimation
            mu_vec=np.array((-np.inf,0,np.inf)) 
            inner=self.distr.cdf(mu_vec[[y.astype('int')+1]]-np.dot(x,parm),0,1)-self.distr.cdf(mu_vec[[y.astype('int')]]-np.dot(x,parm),0,1)
            return -np.sum(np.log(inner))
            
        def estimate(x,y):
            return sp.optimize.minimize(logl,np.array([0,0,0]),args=(x,y),method='bfgs')
        
        self.Bsample_x=np.zeros(self.x.shape)
        self.Bsample_y=np.zeros(self.y.shape)
        
        self.beta_B=np.zeros([niter,self.x.shape[1]])
       
        
        for k in range(niter): # 'niter' is the number of iterations.

            for i in range(len(self.y)):
                j=np.random.uniform(0,len(self.y),1).astype('int') # Transform it into integer is because j is the index.
                self.Bsample_x[i,:]=self.x[j,:]
                self.Bsample_y[i]=self.y[j]
            # By this for-loop, bootstrap samples are constructed.

            self.beta_B[k,:]=estimate(x=self.Bsample_x, y=self.Bsample_y).x 
            # Then, I estimate beta by different bootstrap samples for 'niter' times and store these estimates in a matrix.
        
            
        self.Res = self.beta_B.mean(axis=0) 
        # Get the results of estimation by taking the sample average columns by columns in the bootstrap beta matrix.
        self.std = self.beta_B.std(axis=0)
        # Also can calculate the bootstrap standard errors.
        
        # (1) I construst the 95% CI by normal approximation method.
        self.res=estimate(x=self.x, y=self.y).x # Estimates by using the original dataset.
        self.z_B=self.Res/self.std
        self.ci_B_up=1.96*self.std+self.res # Upper bound of the 95% CI.
        self.ci_B_low=-1.96*self.std+self.res # Lower bound of the 95% CI.
        
        # (2) I calculate the 95% CI of bootstrap by simply taking the 2.5% and 97.5% percentiles.
        obs = np.array([niter*0.025],dtype='i')
        self.ci_B_up1=np.zeros(self.x.shape[1])
        self.ci_B_low1=np.zeros(self.x.shape[1])
        for i in range(self.x.shape[1]):
    
            self.ci_B_up1[i]=np.percentile(self.beta_B[:,i], 97.5)
                              
            self.ci_B_low1[i]=np.percentile(self.beta_B[:,i], 2.5)
        
        # Then, I use the 'if' to choose which method to use.
        
        row=['_cons']
        for i in range(self.x.shape[1]-1):
            row.append('x'+str(i+1))
        # By this for loop, the row indices will be x1,x2,...,xN, where N is the number of explanatory variables.
    
        if method == 'percentile':
            self.data_B = {'coef.':self.Res,'std.errors':self.std,'[0.025':self.ci_B_low1,'0.975]':self.ci_B_up1} 
            return pd.DataFrame(self.data_B, index=row)
        elif method == 'normal':
            self.data_B = {'coef.':self.Res,'std.errors':self.std,'z-value':self.z_B,'[0.025':self.ci_B_low,'0.975]':self.ci_B_up} 
            return pd.DataFrame(self.data_B, index=row)

A brief explanation of the Bootstrap sampling in above class:
- In Bootstrap sampling, let $\hat \beta$ denote the estimated value calculated with the original dataset, $\bar \beta$ denote the Bootstrap-estimated value. Let $\hat \beta_i$ be the value of the statistic from the $i$-th bootstrap sample. 
- Then, $$\hat{se}=\bigg(\frac{1}{B-1} \sum_{i=1}^{B}(\hat \beta_i-\bar \beta)^2\bigg)^{1/2}$$
- where, $$\bar \beta=\frac{1}{B}\sum_{i=1}^{B}\hat \beta_i$$
- The normal-approximation method yields the 95\% confidence intervals $$\bigg[\hat \beta-z_{0.025}\hat{se},\hat \beta+z_{0.025}\hat{se}\bigg]$$ where $z_{0.025}(1.96)$ is the 2.5\% quantile of the standard normal distribution.
- The percentile method yields the confidence intervals $$\bigg[\hat \beta_{0.975}^{*},\hat \beta_{0.025}^{*}\bigg]$$ where $\hat \beta_{p}^{*}$ is the $p$-th quantile of the bootstrap distribution.